In [9]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as plticker
import seaborn as sns
import copy
import math
import random
from SALib.analyze import fast
from SALib.sample import fast_sampler
from SALib.sample import saltelli
from SALib.analyze import sobol
from SALib.test_functions import Ishigami
import pandas as pd

from matplotlib import pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"#"all"
from IPython.core.display import HTML
HTML("""<style>table{font-size: 14px !important;} table td{padding-left: 40px !important;}</style>""")

%run fnn.py
%run import_data





def calc_SA_fast(fnn, N):
    f_num = fnn.feature_num
    STsss = list()
    
    bounds = list()
    maxs = np.zeros(f_num)
    mins = np.zeros(f_num)
    
    for f in range(f_num):
        maxs[f] = max(fnn.x_data[:, f])
        mins[f] = min(fnn.x_data[:, f])
        bounds.append([mins[f], maxs[f]])
    
    for c in range(fnn.class_num):
        problem = {
        'num_vars': f_num,
        'names': [],
        'bounds': bounds
        }
        
        param_values = fast_sampler.sample(problem, N)
        Y = np.zeros(N*f_num, dtype="float64")
        
        for j, X in enumerate(param_values):
            (LL0, LL1, LL2) = fnn.forward(X)
            Y[j] = LL2[0, c]
        
        ss = fast.analyze(problem, Y, M=4, print_to_console=False)
        STsss.append(ss["ST"])
        
    STsss = np.array(STsss).T
    # here now, columns are for relative class (output unit)
    return STsss



def show(ss, cotributions_perc):
    np.std(ss)
    ss = np.hstack((ss, cotributions_perc))
    
    np.savetxt("mydata.csv", ss, header=" ST, cotributions_perc %", delimiter=' , ', fmt="%.5f")
    df = pd.read_csv("mydata.csv", header = 0)
    return df



def get_feature_contributions(fnn, dataname, x, y, x_val, y_val, x_test, y_test):
    
    STsss = calc_SA_fast(fnn, N=fnn.feature_num*64+1)
    
    sum_SA_over_all_of_classes_for_each_factor = np.sum(STsss, axis=1, keepdims=True) # each row
    ss = sum_SA_over_all_of_classes_for_each_factor # ss dimens : (hidd_num, 1)
    
    total = np.sum(ss)
    cotributions = ss/total
    
    cotributions_perc = cotributions*100
    
    
    return fnn, ss, cotributions_perc








def show_cee():
    with open("cee.txt", "r") as ff:
        lines = ff.readlines()
        new_lines = np.zeros(len(lines)*3, dtype="float64").reshape(len(lines), 3)

        for i,line in enumerate(lines):
            new_lines[i, 0] = float(line.split(",")[0])
            new_lines[i, 1] = float(line.split(",")[1])
            new_lines[i, 2] = float(line.split(",")[2])

    y1 = new_lines[:, 0]
    y2 = new_lines[:, 1]
    y3 = new_lines[:, 2]

    x = np.arange(0, y1.shape[0])

    fig = plt.figure(figsize=(11,8), dpi=120)
    ax1 = fig.add_subplot(111)

    ax1.plot(x, y1, 'y', label=1, linewidth=0.7)
    ax1.plot(x, y2, 'b', label=2, linewidth=0.7)
    ax1.plot(x, y3, 'g', label=3, linewidth=0.7)

    plt.legend(['y1 = cee_data', 'y2 = cee_val', 'y3 = cee_test'], loc='upper left')
    sns.set_style("whitegrid")
    ax1.grid(False)
    

#    plt.savefig("stuff/results/" + lines0[0] +'/cee_all.png')
    
    plt.show()
    plt.close("all")









In [ ]:
#### weights initialized with the same random seed.

import matplotlib.pyplot as plt
import matplotlib.ticker as plticker
import seaborn as sns
%run fnn.py
%run import_data



dataname="waveform"

(x, y, x_val, y_val, x_test, y_test) = get_data(dataname)

fnn = FnnSingleLayer(hidd_num=20, print_every=10)
fnn.train(x, y, x_val, y_val, x_test, y_test, epochs=1000, learn_rate=0.1, patience=5, new=True, dontshow=False)


(fnn, ss, cotributions_perc) = get_feature_contributions(fnn, dataname, x, y, x_val, y_val, x_test, y_test)
show(ss, cotributions_perc)
# show_cee()

sorted_indexes = np.zeros(fnn.feature_num, dtype="int64")
cc = copy.deepcopy(cotributions_perc)
for i in range(fnn.feature_num):
    indx = np.argmin(cc)
    sorted_indexes[i] = np.where(cc[indx] == cotributions_perc)[0][0]
    cc = np.delete(cc, indx, 0)


new_x = copy.deepcopy(x[:, sorted_indexes[0]].reshape(x.shape[0], 1))
new_x_val = copy.deepcopy(x_val[:, sorted_indexes[0]].reshape(x_val.shape[0], 1))
new_x_test = copy.deepcopy(x_test[:, sorted_indexes[0]].reshape(x_test.shape[0], 1))
val_accs_as = np.zeros(fnn.feature_num, dtype="float64")
test_accs_as = np.zeros(fnn.feature_num, dtype="float64")
val_accs_de = np.zeros(fnn.feature_num, dtype="float64")
test_accs_de = np.zeros(fnn.feature_num, dtype="float64")





for i,item in enumerate(sorted_indexes):
    
    deleting_features = sorted_indexes[i+1:]
    
    new_x = np.delete(x, deleting_features, 1)
    new_x_val = np.delete(x_val, deleting_features, 1)
    new_x_test = np.delete(x_test, deleting_features, 1)
    
    fnn_fk = copy.deepcopy(fnn)    
    
    nodes = deleting_features
    fnn_fk.train(new_x, y, new_x_val, y_val, new_x_test, y_test, epochs=100, learn_rate=0.1, patience=5, new=True, dontshow=True)
#     fnn_fk.prune_input_nodes(new_x, new_x_val, new_x_test, nodes)
    
    
    val_accs_as[i] = fnn_fk.calc_acc(new_x_val, y_val)
    test_accs_as[i] = fnn_fk.calc_acc(new_x_test, y_test)




new_x = copy.deepcopy(x[:, sorted_indexes[0]].reshape(x.shape[0], 1))
new_x_val = copy.deepcopy(x_val[:, sorted_indexes[0]].reshape(x_val.shape[0], 1))
new_x_test = copy.deepcopy(x_test[:, sorted_indexes[0]].reshape(x_test.shape[0], 1))
reversed_sorted_indexes = np.flip(sorted_indexes, 0)
    
    
for i,item in enumerate(reversed_sorted_indexes):
    deleting_features = reversed_sorted_indexes[i+1:]
    
    new_x = np.delete(x, deleting_features, 1)
    new_x_val = np.delete(x_val, deleting_features, 1)
    new_x_test = np.delete(x_test, deleting_features, 1)
    
    fnn_fk = copy.deepcopy(fnn)    
    
    nodes = deleting_features
    fnn_fk.train(new_x, y, new_x_val, y_val, new_x_test, y_test, epochs=100, learn_rate=0.1, patience=5, new=True, dontshow=True)
#     fnn_fk.prune_input_nodes(new_x, new_x_val, new_x_test, nodes)

    
    val_accs_de[i] = fnn_fk.calc_acc(new_x_val, y_val)
    test_accs_de[i] = fnn_fk.calc_acc(new_x_test, y_test)




In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as plticker
# plt.rcParams.update(plt.rcParamsDefault)


def show_accs(x1, y1, x2, y2, xx1, yy1, xx2, yy2):
    fig = plt.figure(figsize=(16,14), dpi=270)
    ax1 = fig.add_subplot(211)

    ax1.plot(x1, y1, c="k", markerfacecolor="#3498DB", markeredgecolor="#3498DB", label="Validation", linewidth=0.5, linestyle='-', markersize=4.5, marker='s')
    ax1.plot(x2, y2, c="k", markerfacecolor="#D35400", markeredgecolor="#D35400", label="Test", linewidth=0.5, linestyle='-', markersize=6.5, marker='*')

    
    plt.xlabel('Features \n (a) \n\n', fontsize=15)
    plt.ylabel('Classification accuracy (%)', fontsize=15)
    plt.title('Accuracy while adding each feature (ascending)', fontsize=15)
    
    
    
    ax1.grid(which='major', linestyle='--', dashes=(1, 5), linewidth='1', color='#595959')
    ax1.grid(which='minor', linestyle="--", linewidth='0.3', color="#cccccc")
    ax1.set_axisbelow(True)
    ax1.set_xlim(1, x.shape[1])
    ax1.xaxis.set_major_locator(plticker.MultipleLocator(base=1))
    
    plt.legend(loc=2, prop={'size': 12})
    
    
    
    # ------------------------------------------------------------------------------------
    ax2 = fig.add_subplot(212)

    ax2.plot(xx1, yy1, c="k", markerfacecolor="#3498DB", markeredgecolor="#3498DB", label="Validation", linewidth=0.5, linestyle='-', markersize=4.5, marker='s')
    ax2.plot(xx2, yy2, c="k", markerfacecolor="#D35400", markeredgecolor="#D35400", label="Test", linewidth=0.5, linestyle='-', markersize=6.5, marker='*')
    
    plt.xlabel('Features \n (b)', fontsize=15)
    plt.ylabel('Classification accuracy (%)', fontsize=15)
    plt.title('Accuracy while adding each feature (descending)', fontsize=15)
    
    ax2.grid(which='major', linestyle='--', dashes=(1, 5), linewidth='1', color='#595959')
    ax2.grid(which='minor', linestyle="--", linewidth='0.3', color="#cccccc")
    ax2.set_axisbelow(True)
    ax2.set_xlim(1, x.shape[1])
    ax2.xaxis.set_major_locator(plticker.MultipleLocator(base=1))
    
    sns.set_style("whitegrid", {"legend.fancybox":True})
    plt.legend(loc=2, prop={'size': 12})
    plt.rcParams["font.family"] = "Times New Roman"
    
    
    plt.show()




xx = np.arange(1, x.shape[1]+1)
show_accs(xx, val_accs_as, xx, test_accs_as, xx, val_accs_de, xx, test_accs_de)


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns



my_range = np.arange(21, dtype="int64") + 1


fig = plt.figure(figsize=(8,10), dpi=120)
ax1 = fig.add_subplot(111)


ax1.hlines(y=my_range, xmin=0, xmax=cotributions_perc, color='skyblue')
ax1.plot(cotributions_perc, my_range, "o")


ax1.set_yticks(my_range, ordered_df['group'].all())
ax1.set_title("Saliency of features for " + dataname + " dataset", loc='left')
ax1.set_xlabel('Contribution (%)')
ax1.set_ylabel('No. of feature')
ax1.set_ylim(0.5, 21.5)
ax1.set_xlim(0, 12)

ax1.grid(linestyle="-", linewidth=1, color="#ffffff")
sns.set_style("dark")


ax1.yaxis.set_major_locator(plticker.MultipleLocator(base=1))


plt.yticks(np.arange(min(my_range), max(my_range)+1, 1.0))

plt.rcParams['font.size'] = 12

plt.show()
plt.close("all")






In [46]:
# preparing mushroom dataset :
# and coverting categorical values to numeric by label encoding!
# http://pbpython.com/categorical-encoding.html


import pandas as pd
import numpy as np



df = pd.read_csv("data/mushrooms.txt", na_values="?" )
df.head()

df = df.dropna() # Drop all rows that contain null values

# df["cap-shape"].value_counts()


headers = np.array(df.columns)

# df[header] = df[header].cat.codes # it only works with cat dtype!



for header in headers:
    # df[header].value_counts()
    df[header] = df[header].astype('category')
    df[header] = df[header].cat.codes


df = df.drop('veil-type', 1)

df.head()
df.iloc[:, 1:].to_csv('data/mushrooms_data.csv', index=False, header=False)
df.iloc[:, 0].to_csv('data/mushrooms_targets.csv', index=False, header=False)


,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,1,5,2,4,1,6,1,0,1,2,...,2,2,5,5,0,1,3,1,3,5
1,0,5,2,7,1,0,1,0,0,2,...,2,2,5,5,0,1,3,2,2,1
2,0,0,2,6,1,3,1,0,0,3,...,2,2,5,5,0,1,3,2,2,3
3,1,5,3,6,1,6,1,0,1,3,...,2,2,5,5,0,1,3,1,3,5
4,0,5,2,3,0,5,1,1,0,2,...,2,2,5,5,0,1,0,2,0,1


In [ ]:
# (df - df.mean()) / (df.max() - df.min())